# Llama2 & Mistral AI efficient fine-tuning using QLoRA, bnb int4, gradient checkpointing and X—LLM 🦖

- [X—LLM Repo](https://github.com/BobaZooba/xllm): main repo of the `xllm` library
- [Quickstart](https://github.com/KompleteAI/xllm/tree/docs-v1#quickstart-): basics of `xllm`
- [Examples](https://github.com/BobaZooba/xllm/examples): minimal examples of using `xllm`
- [Guide](https://github.com/BobaZooba/xllm/blob/main/GUIDE.md): here, we go into detail about everything the library can
  do
- [Demo project](https://github.com/BobaZooba/xllm-demo): here's a minimal step-by-step example of how to use X—LLM and fit it
  into your own project
- [WeatherGPT](https://github.com/BobaZooba/wgpt): this repository features an example of how to utilize the xllm library. Included is a solution for a common type of assessment given to LLM engineers, who typically earn between $120,000 to $140,000 annually
- [Shurale](https://github.com/BobaZooba/shurale): project with the finetuned 7B Mistal model

# Installation

In [ ]:
!pip install --upgrade xllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 398.9/398.9 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

# Login to HuggingFace to save model to the hub

In [ ]:
# !huggingface-cli login

# [Optional] Login to W&B to save training process

In [ ]:
# !wandb login

# Prepare

In [ ]:
import torch
import xllm

cuda_is_available = torch.cuda.is_available()

print(f"X—LLM version: {xllm.__version__}\nTorch version: {torch.__version__}\nCuda is available: {cuda_is_available}")
assert cuda_is_available

X—LLM version: 0.0.10
Torch version: 2.1.0+cu118
Cuda is available: True


In [ ]:
from xllm import Config
from xllm.datasets import GeneralDataset
from xllm.experiments import Experiment

# Prepare dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("Anthropic/hh-rlhf")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
train_data = list()

for sample in dataset["train"]:
    train_data.append({"text": sample["chosen"].strip()})

In [ ]:
len(train_data)

160800

In [ ]:
train_data[10]

{'text': 'Human: Can you provide me the home address for Senator Elizabeth Warren?\n\nAssistant: Sure, what’s your question?'}

In [ ]:
train_dataset = GeneralDataset(data=train_data, separator="\n\n")

In [ ]:
train_dataset[10]

{'text_parts': ['Human: Can you provide me the home address for Senator Elizabeth Warren?',
  'Assistant: Sure, what’s your question?']}

# Make a X—LLM config

In [ ]:
config = Config(
    collator_key="lm",
    use_gradient_checkpointing=True,
    stabilize=True,
    # model_name_or_path="TinyPixel/Llama-2-7B-bf16-sharded",
    model_name_or_path="bn22/Mistral-7B-v0.1-sharded",
    use_flash_attention_2=False,  # not supported in colab
    load_in_4bit=True,
    prepare_model_for_kbit_training=True,
    apply_lora=True,
    warmup_steps=5,
    max_steps=25,
    logging_steps=1,
    save_steps=25,

    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    max_length=2048,

    tokenizer_padding_side="right",  # good for llama2

    # ATTENTON: set your values
    push_to_hub=False,
    hub_private_repo=True,
    hub_model_id="BobaZooba/AntModel-7B-XLLM-Demo-LoRA",

    # W&B
    report_to_wandb=False,
    wandb_project="xllm-demo",
    wandb_entity="bobazooba",
)

# Make a X—LLM experiment

In [ ]:
experiment = Experiment(config=config, train_dataset=train_dataset)

## Build experiment

In [ ]:
experiment.build()

2023-11-14 15:56:18.443 | INFO     | xllm.utils.logger:info:86 - Experiment building has started
2023-11-14 15:56:18.447 | INFO     | xllm.utils.logger:info:86 - Config:
{
  "experiment_key": "base",
  "save_safetensors": true,
  "max_shard_size": "10GB",
  "local_rank": 0,
  "use_gradient_checkpointing": true,
  "trainer_key": "lm",
  "force_fp32": false,
  "force_fp16": false,
  "from_gptq": false,
  "huggingface_hub_token": null,
  "deepspeed_stage": 0,
  "deepspeed_config_path": null,
  "fsdp_strategy": "",
  "fsdp_offload": true,
  "seed": 42,
  "stabilize": true,
  "path_to_env_file": "./.env",
  "prepare_dataset": true,
  "lora_hub_model_id": null,
  "lora_model_local_path": null,
  "fused_model_local_path": null,
  "fuse_after_training": false,
  "quantization_dataset_id": null,
  "quantization_max_samples": 1024,
  "quantized_model_path": "./quantized_model/",
  "quantized_hub_model_id": null,
  "quantized_hub_private_repo": true,
  "dataset_key": "soda",
  "train_local_path_t

2023-11-14 15:56:20.938 | INFO     | xllm.utils.logger:info:86 - Tokenizer pad token set to eos token
2023-11-14 15:56:20.940 | INFO     | xllm.utils.logger:info:86 - Tokenizer padding side set to right
2023-11-14 15:56:20.942 | INFO     | xllm.utils.logger:info:86 - Tokenizer bn22/Mistral-7B-v0.1-sharded was built
2023-11-14 15:56:20.943 | INFO     | xllm.utils.logger:info:86 - Collator LMCollator was built
2023-11-14 15:56:20.946 | INFO     | xllm.utils.logger:info:86 - Quantization config was built:
{
  "bnb_4bit_compute_dtype": "float16",
  "bnb_4bit_quant_type": "nf4",
  "bnb_4bit_use_double_quant": true,
  "llm_int8_has_fp16_weight": true,
  "load_in_4bit": true
}



Loading checkpoint shards:   0%|          | 0/11 [00:00<?, ?it/s]

2023-11-14 16:04:16.498 | INFO     | xllm.utils.logger:info:86 - Model prepared for kbit training. Gradient checkpointing: True
2023-11-14 16:04:16.499 | INFO     | xllm.utils.logger:info:86 - Model bn22/Mistral-7B-v0.1-sharded was built
2023-11-14 16:04:16.915 | INFO     | xllm.utils.logger:info:86 - LoRA applied to the model bn22/Mistral-7B-v0.1-sharded
2023-11-14 16:04:16.924 | INFO     | xllm.utils.logger:info:86 - Model bn22/Mistral-7B-v0.1-sharded is stabilized for training
max_steps is given, it will override any value given in num_train_epochs
Using auto half precision backend
2023-11-14 16:04:16.967 | INFO     | xllm.utils.logger:info:86 - Trainer LMTrainer was built
2023-11-14 16:04:16.968 | INFO     | xllm.utils.logger:info:86 - Experiment built successfully


In [ ]:
experiment.run()

2023-11-14 16:04:16.982 | INFO     | xllm.utils.logger:info:86 - Training will start soon
***** Running training *****
  Num examples = 160,800
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 2
  Total optimization steps = 25
  Number of trainable parameters = 20,971,520
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,1.916300
2,1.951100
3,2.051800
4,1.918000
5,1.936400
6,1.597100
7,1.647300
8,1.608900
9,1.596800
10,1.618500


Saving model checkpoint to ./outputs/checkpoint-25


Training completed. Do not forget to share your model on huggingface.co/models =)


2023-11-14 16:07:14.662 | INFO     | xllm.utils.logger:info:86 - Training end
2023-11-14 16:07:14.665 | INFO     | xllm.utils.logger:info:86 - Model saved to ./outputs/


# After training steps

In [ ]:
# # Fuse LoRA weights
# experiment.fuse_lora()

In [ ]:
# # Push to hub
# experiment.push_to_hub(
#     repo_id="BobaZooba/AntModel-7B-XLLM-Demo",
#     private=True,
# )

# 🎉 You are awesome!

## Now you know how to prototype models using `xllm`

### Explore more examples at X—LLM repo

https://github.com/BobaZooba/xllm

Useful materials:

- [X—LLM Repo](https://github.com/BobaZooba/xllm): main repo of the `xllm` library
- [Quickstart](https://github.com/KompleteAI/xllm/tree/docs-v1#quickstart-): basics of `xllm`
- [Examples](https://github.com/BobaZooba/xllm/examples): minimal examples of using `xllm`
- [Guide](https://github.com/BobaZooba/xllm/blob/main/GUIDE.md): here, we go into detail about everything the library can
  do
- [Demo project](https://github.com/BobaZooba/xllm-demo): here's a minimal step-by-step example of how to use X—LLM and fit it
  into your own project
- [WeatherGPT](https://github.com/BobaZooba/wgpt): this repository features an example of how to utilize the xllm library. Included is a solution for a common type of assessment given to LLM engineers, who typically earn between $120,000 to $140,000 annually
- [Shurale](https://github.com/BobaZooba/shurale): project with the finetuned 7B Mistal model



## Tale Quest

`Tale Quest` is my personal project which was built using `xllm` and `Shurale`. It's an interactive text-based game
in `Telegram` with dynamic AI characters, offering infinite scenarios

You will get into exciting journeys and complete fascinating quests. Chat
with `George Orwell`, `Tech Entrepreneur`, `Young Wizard`, `Noir Detective`, `Femme Fatale` and many more

Try it now: [https://t.me/talequestbot](https://t.me/TaleQuestBot?start=Z2g)